# 실험 03-01 다이오드 리미터 회로 실험

*******************************************************************************
## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1-1 결과 분석 (실험 과정 5, 6)

### 회로도

* 병렬형 양 리미터 

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_01_01_01.jpeg)
![Markdown-mark](../experiment/circuit_01_01_02.jpeg)

<!-- ![Markdown-mark](../experiment/m_01_01.png) -->
<!-- ![Markdown-mark](../experiment/m_01_02.png) -->
<!-- ![Markdown-mark](../experiment/m_01_03.png) -->

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01_05.csv'))

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = []
vss = []
vos = []
labels = ['1V', '2V', '3V', '4V', '5V']

for idx in range(len(labels)):
    times.append(data[idx]['TIME'] * 1000)
    vss.append(data[idx]['Sample CH1'] * 1000)
    vos.append(data[idx]['Sample CH2'] * 1000)
    vss[-1] = savgol_filter(vss[-1], 200, 2)
    vos[-1] = savgol_filter(vos[-1], 200, 2)


### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.4, 4)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
idx = 0

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

In [ ]:
idx = 4

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

### 4. 출력 전압의 리미팅 레벨을 구한다.

In [ ]:
V_OUT_limit_mV = []

for idx in range(len(labels)):
    V_OUT_limit_mV.append(vos[idx].max())

print("VB = ", end='')
print(labels)
print_array("V_OUT_limit_mV", V_OUT_limit_mV)

### 5. 입력 전압을 x축, 출력 전압 y축으로 하는 입출력 전달 특성 그래프를 그린다.

In [ ]:
plt.xlabel('VS_mV')
plt.ylabel('VO_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-5500, 5500)
plt.ylim(-5500, 5500)
for idx in range(len(labels)):
    draw_plot(vss[idx], vos[idx], 'VB=%s' % labels[idx], idx, scatter=True)
plt.show()

*******************************************************************************
## 실험 1-2 결과 분석 (실험 과정 7)

### 회로도

* 병렬형 음 리미터 

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_01_02_01.jpeg)
![Markdown-mark](../experiment/circuit_01_02_02.jpeg)

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_05.csv'))

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = []
vss = []
vos = []
labels = ['1V', '2V', '3V', '4V', '5V']

for idx in range(len(labels)):
    times.append(data[idx]['TIME'] * 1000)
    vss.append(data[idx]['Sample CH1'] * 1000)
    vos.append(data[idx]['Sample CH2'] * 1000)
    vss[-1] = savgol_filter(vss[-1], 200, 2)
    vos[-1] = savgol_filter(vos[-1], 200, 2)


### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.4, 4)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
idx = 0

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

### 4. 출력 전압의 리미팅 레벨을 구한다.

In [ ]:
V_OUT_limit_mV = []

for idx in range(len(labels)):
    V_OUT_limit_mV.append(vos[idx].min())

print("VB = ", end='')
print(labels)
print_array("V_OUT_limit_mV", V_OUT_limit_mV)

### 5. 입력 전압을 x축, 출력 전압 y축으로 하는 입출력 전달 특성 그래프를 그린다.

In [ ]:
plt.xlabel('VS_mV')
plt.ylabel('VO_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-5500, 5500)
plt.ylim(-5500, 5500)
for idx in range(len(labels)):
    draw_plot(vss[idx], vos[idx], 'VB=%s' % labels[idx], idx, scatter=True)
plt.show()

*******************************************************************************
## 실험 2-1 결과 분석 (실험 과정 5, 6)

### 회로도

* 직렬형 양 리미터 

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_02_01_01.jpeg)
![Markdown-mark](../experiment/circuit_02_01_02.jpeg)

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01_05.csv'))

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = []
vss = []
vos = []
labels = ['1V', '2V', '3V', '4V', '5V']

for idx in range(len(labels)):
    times.append(data[idx]['TIME'] * 1000)
    vss.append(data[idx]['Sample CH1'] * 1000)
    vos.append(data[idx]['Sample CH2'] * 1000)
    vss[-1] = savgol_filter(vss[-1], 200, 2)
    vos[-1] = savgol_filter(vos[-1], 200, 2)


### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.4, 4)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
idx = 0

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

In [ ]:
idx = 4

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

In [ ]:
idx = 3

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
vdiff = vos[idx] - vss[idx]
draw_plot(times[idx], vdiff, 'VB=%s V(VDIFF)' % labels[idx], 2)
plt.show()

print_value('Maximum value of V(VDIFF) (mV)', vdiff.max())

### 4. 출력 전압의 리미팅 레벨을 구한다.

In [ ]:
V_OUT_limit_mV = []

for idx in range(len(labels)):
    V_OUT_limit_mV.append(vos[idx].max())

print("VB = ", end='')
print(labels)
print_array("V_OUT_limit_mV", V_OUT_limit_mV)

### 5. 입력 전압을 x축, 출력 전압 y축으로 하는 입출력 전달 특성 그래프를 그린다.

In [ ]:
plt.xlabel('VS_mV')
plt.ylabel('VO_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-5500, 5500)
plt.ylim(-5500, 5500)
for idx in range(len(labels)):
    draw_plot(vss[idx], vos[idx], 'VB=%s' % labels[idx], idx, scatter=True)
plt.show()

*******************************************************************************
## 실험 2-2 결과 분석 (실험 과정 7)

### 회로도

* 직렬형 음 리미터 

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_02_02_01.jpeg)
![Markdown-mark](../experiment/circuit_02_02_02.jpeg)

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_05.csv'))

### 2. 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = []
vss = []
vos = []
labels = ['1V', '2V', '3V', '4V', '5V']

for idx in range(len(labels)):
    times.append(data[idx]['TIME'] * 1000)
    vss.append(data[idx]['Sample CH1'] * 1000)
    vos.append(data[idx]['Sample CH2'] * 1000)
    vss[-1] = savgol_filter(vss[-1], 200, 2)
    vos[-1] = savgol_filter(vos[-1], 200, 2)


### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-0.4, 4)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
for idx in range(len(labels)):
    draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], idx, color_idx=0)
for idx in range(len(labels)):
    draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], idx, color_idx=1)
plt.show()

In [ ]:
idx = 0

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

In [ ]:
idx = 4

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
plt.show()

In [ ]:
idx = 1

plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(0.9, 2.1)
plt.ylim(-6000, 6000)
draw_plot(times[idx], vss[idx], 'VB=%s V(VS)' % labels[idx], 0)
draw_plot(times[idx], vos[idx], 'VB=%s V(VO)' % labels[idx], 1)
vdiff = vos[idx] - vss[idx]
draw_plot(times[idx], vdiff, 'VB=%s V(VDIFF)' % labels[idx], 2)
plt.show()

print_value('Miniimum value of V(VDIFF) (mV)', vdiff.min())

### 4. 출력 전압의 리미팅 레벨을 구한다.

In [ ]:
V_OUT_limit_mV = []

for idx in range(len(labels)):
    V_OUT_limit_mV.append(vos[idx].min())

print("VB = ", end='')
print(labels)
print_array("V_OUT_limit_mV", V_OUT_limit_mV)

### 5. 입력 전압을 x축, 출력 전압 y축으로 하는 입출력 전달 특성 그래프를 그린다.

In [ ]:
plt.xlabel('VS_mV')
plt.ylabel('VO_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim(-5500, 5500)
plt.ylim(-5500, 5500)
for idx in range(len(labels)):
    draw_plot(vss[idx], vos[idx], 'VB=%s' % labels[idx], idx, scatter=True)
plt.show()